In [46]:
from collections import namedtuple, deque
import numpy as np

In [33]:
Point = namedtuple("Point", ["x", "y"])

class HeightMap:
    def __init__(self, filename):
        self.filename = filename
        self.grid = self._build_grid(filename)
        self.height = self.grid.shape[0]
        self.width  = self.grid.shape[1]

    def _build_grid(self, filename):
        data = []
        with open(filename, "r") as fh:
            for l in fh:
                data.append([int(c) for c in l.strip()])
        return np.array(data, dtype=int)

    def get(self, p):
        return self.grid[p.y, p.x]
    
    def neighbors(self, p):
        out = []
        for np in [Point(p.x,p.y-1), Point(p.x+1,p.y), Point(p.x,p.y+1), Point(p.x-1,p.y)]:
            if np.x >= 0 and np.x < self.width and np.y >= 0 and np.y < self.height:
                yield np

    def print_grid(self):
        for j in range(self.height):
            for i in range(self.width):
                p = Point(i,j)
                print(self.get(p), end="")
            print()
                
test = HeightMap("test.txt")
test.print_grid()
print(test.height)
print(test.width)
print(list(test.neighbors(Point(0, 0))))
print(list(test.neighbors(Point(1, 1))))
print(list(test.neighbors(Point(4, 4))))

2199943210
3987894921
9856789892
8767896789
9899965678
5
10
[Point(x=1, y=0), Point(x=0, y=1)]
[Point(x=1, y=0), Point(x=2, y=1), Point(x=1, y=2), Point(x=0, y=1)]
[Point(x=4, y=3), Point(x=5, y=4), Point(x=3, y=4)]


In [42]:
def calc_low_points(hm, debug=False):
    mins = []
    risk = 0
    for j in range(hm.height):
        for i in range(hm.width):
            p = Point(i,j)
            is_min = True
            for np in hm.neighbors(p):
                if hm.get(np) <= hm.get(p):
                    is_min = False
            if is_min:
                mins.append(p)
                risk += hm.get(p) + 1
    if debug: print(mins)
    return mins, risk

calc_low_points(test)

[Point(x=1, y=0), Point(x=9, y=0), Point(x=2, y=2), Point(x=6, y=4)]


([Point(x=1, y=0), Point(x=9, y=0), Point(x=2, y=2), Point(x=6, y=4)], 15)

In [44]:
inp = HeightMap("input.txt")
calc_low_points(inp)[1]

532

In [45]:
def print_points(m, pts):
    for j in range(m.height):
        for i in range(m.width):
            p = Point(i,j)
            if p in pts:
                print("#", end="")
            else:
                print(m.get(p), end="")
        print()
        
print_points(test, calc_low_points(test)[0])

2#9994321#
3987894921
98#6789892
8767896789
989996#678


In [61]:
def find_basin(m, pmin):
    """Use BFS from the minimum point to find all points in the basin."""
    q = deque()
    q.append(pmin)
    basin = set()
    while q:
        p = q.popleft()
        pv = m.get(p)
        basin.add(p)
        #print(p, pv)
        for np in m.neighbors(p):
            npv = m.get(np)
            #print(np, npv)
            if np not in basin and npv >= pv and npv != 9:
                q.append(np)
    return basin

for pmin in calc_low_points(test)[0]:
    basin = find_basin(test, pmin)
    print("{} {}".format(pmin, len(basin)))
    print_points(test, basin)

Point(x=1, y=0) 3
##99943210
#987894921
9856789892
8767896789
9899965678
Point(x=9, y=0) 9
21999#####
398789#9##
985678989#
8767896789
9899965678
Point(x=2, y=2) 14
2199943210
39###94921
9#####9892
#####96789
9#99965678
Point(x=6, y=4) 9
2199943210
3987894921
9856789#92
876789###9
98999#####


In [67]:
def find_biggest_basins(m):
    bsizes = []
    for pmin in calc_low_points(m)[0]:
        basin = find_basin(m, pmin)
        bsizes.append(len(basin))
    largest = sorted(bsizes, reverse=True)[0:3]
    print(largest)
    return largest[0] * largest[1] * largest[2]

find_biggest_basins(test)

[14, 9, 9]


1134

In [68]:
find_biggest_basins(inp)

[110, 102, 99]


1110780